In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np   
import matplotlib.pyplot as plt
import seaborn as sns
import random

from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils
from keras.optimizers import SGD

from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import base64

import tensorflow as tf
print(tf.__version__)

In [ ]:
#Importar dados
celeba = '../input/celeba-dataset/'
imagem = celeba + 'img_align_celeba/img_align_celeba/'

exemplo_img = imagem + '000508.jpg'

training = 10000
test = 5000
validation = 5000

In [ ]:
base_new = pd.read_csv(celeba + 'list_attr_celeba.csv')
base_new.set_index('image_id', inplace=True)

In [ ]:
#Apresentar o shape
base_new.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
base_new.shape

In [ ]:
#Gráfico apresentando a distribuição de homens e mulheres

plt.title('Quantidade de Homens e Mulheres')
sns.countplot(x='Male', data=base_new, color="g")
plt.show()

In [ ]:
base_new.head()

In [ ]:
base_new.shape

In [ ]:
#Plotar um exemplo

img = load_img(exemplo_img)
plt.imshow(img)
base_new.loc[exemplo_img.split('/')[-1]][['Smiling','Male','Young']] #apresentando os atributos.

In [ ]:
#Importando a base partição
particao = pd.read_csv(celeba + 'list_eval_partition.csv')
#Indexando a base
particao.set_index('image_id', inplace=True)
particao.head()

In [ ]:
particao.shape

In [ ]:
#Apresentando os dados de treino, validação e teste 
particao['partition'].value_counts().sort_index()

In [ ]:
#Fazer um join com os dados Male
base = particao.join(base_new['Male'], how='inner')
base.head()

In [ ]:
#transformar a imagem em array e normalizar os dados.
def load_reshape_img(dname):
    img = load_img(dname)
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x

In [ ]:
x

In [ ]:
def generate(partition, atri, num_sa):
    '''
    partition
        0 -> train
        1 -> validation
        2 -> test
    
    '''
    
    cb = base[(base['partition'] == partition) 
                           & (base[atri] == 0)].sample(int(num_sa/2))
    cb = pd.concat([cb,
                      base[(base['partition'] == partition) 
                                  & (base[atri] == 1)].sample(int(num_sa/2))])

    #treino e validação
    if partition != 2:
        xi = np.array([load_reshape_img(imagem + dname) for dname in cb.index])
        xi = xi.reshape(xi.shape[0], 218, 178, 3)
        yu = np_utils.to_categorical(cb[atri],2)
    # for Test
    else:
        xi = []
        yu = []

        for index, target in df_.iterrows():
            im = cv2.imread(imagem + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            xi.append(im)
            yu.append(target[atri])

    return xi, yu

In [ ]:
#Gerar dados
datagen =  ImageDataGenerator(rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)

In [ ]:
x_train, y_train = generate(0, 'Male', training)

# x_Train - preparar os dados
train_dados =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_dados.fit(x_train)

train_generator = train_dados.flow(
x_train, y_train, batch_size=batch_size,)

In [ ]:
x_valid, y_valid = generate(1, 'Male', validation)

valid_dados = ImageDataGenerator(
  preprocessing_function=preprocess_input,
)

valid_dados.fit(x_valid)

validation_generator = valid_dados.flow(
x_valid, y_valid,
)

In [ ]:
#Construção e treinamento do modelo

from tensorflow.keras import layers, models

x_train = models.Sequential()

x_train.add(layers.GlobalAveragePooling2D())
x_train.add(layers.MaxPooling2D(2,2))

x_train.add(layers.Dense(512, activation='relu'))

x_train.add(layers.Dense(2, activation='softmax'))

In [ ]:
#x_train.summary()


x_train.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#Treinamento do modelo

epochs = 50
history = x_train.fit(x_train, batch_size=, epochs=epochs)

In [ ]:
avaliacao = x_train.evaluate(x_train, y_train)
avaliacao

In [ ]:
predicted_classes = x_train.predict_classes(x_train)
predicted_classes

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, predicted_classes)
plt.figure(figsize=(14,10))
sns.heatmap(cm, annot=True) 

In [ ]:
from sklearn.metrics import classification_report
num_classes = 10
target_names = ["Class {}".format(i) for i in range(num_classes)]
target_names